In [40]:
"""Code to scrape text off of WTA website"""

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sn
import re
import math

In [5]:
def collect_hikeurls(starturl):
    """Collecting all of the websites for all of the hikes"""
    hike_links = []
    path = starturl
    counter = 0
    while True:
        r = requests.get(path)
        soup = BeautifulSoup(r.text, 'lxml')
        for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
            hike_links.append(div['href'])
            counter += 1
        print ('Collected %d websites' % counter)
        link = soup.find('span', attrs={'class': 'next'})
        if link is None:
            break
        else:
            path = link.a['href']
    return hike_links


def parser(url):
    """Parses URL into hiking dataset.
    Parser collects html soup and populates
    washington_hikes table with relevant data. Data cleaning is completed in a
    seperate python script."""

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    row_data = {}
    row_data['hike_name'] = soup.find('h1', attrs={'class': "documentFirstHeading"}).text.strip()
    try:
        row_data['region'] = soup.find('div', attrs={'class': "hike-stat grid_3 alpha"}).div.text.strip()
    except:
        try:
            row_data['region'] = soup.find('div', attrs={'id': 'hike-region'}).span.text
        except:
            row_data['region'] = 'NR'
    try:
        lengain = []
        for div in soup.findAll('div', attrs={'class': "hike-stat grid_3"}):
            lengain.append(div.div.span.text)
        row_data['length'] = lengain[0]
        row_data['elevation gain'] = lengain[1]
    except:
        row_data['length'] = 'NR'
        row_data['elevation gain'] = 'NR'
    try:
        row_data['rating'] = soup.find('div', attrs={'class': "current-rating"}).text
    except:
        row_data['rating'] = 'NR'
    try:
        row_data['number_votes'] = soup.find('div', attrs={'class': "rating-count"}).text
    except:
        row_data['number_votes'] = 'NR'
    try:
        features = []
        for div in soup.findAll('div', attrs={'class': "feature grid_1 "}):
            features.append(div['data-title'])
        row_data['features'] = features
    except:
        row_data['features'] = 'NR'
    try:
        row_data['which_pass'] = soup.find('div', attrs={'id': "pass-required-info"}).a.text
    except:
        row_data['which_pass'] = 'NR'
    try:
        row_data['latlong'] = soup.find('a', attrs={'class': "visualNoPrint full-map"})['href']
    except:
        row_data['latlong'] = 'NR'
    try:
        row_data['numReports'] = soup.find('span', attrs={'class': 'ReportCount'}).text
    except:
        row_data['numReports'] = 'NR'
    row_data['url'] = url
    return row_data


def build_dataset(data, urls):
    """Adds parsed fields to dataset.
    Runs though links in urls and applies parser function to the soup Collected
    from the link.  Saves to csv every 50 querries.
    data: dataset to append data to
    urls: List of urls"""
    cnter = 0
    row = -1
    for lnk in urls:
        row += 1
        d = parser(lnk)
        for key in d.keys():
            data.set_value(row, key, d[key])
        cnter += 1
        if cnter % 50 == 0:
            data.to_csv('washington_hikes.csv', encoding='utf-8')
            print ('Saving ', cnter)
    data.to_csv('washington_hikes.csv', encoding='utf-8')


def getting_hike_desc(url):
    """Collects hike description from url."""
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        return soup.find('div', attrs={'id': 'hike-body-text'}).p.text
    except:
        return None



def getHikeDescription(data, row=-1, counter=0):
    """Collects hike description by row of a dataframe.
    Keeps track of number of descriptions collected.
    Can specify starting row in hyperparameters.
    data: dataset to add description to
    row: row to start on
    counter: counter to keep track of descriptions collected"""

    for url in data['url']:
        row += 1
        data.set_value(row, 'hike_desc', getting_hike_desc(data['url'][row]))
        counter += 1
        if counter % 50 == 0:
            data.to_csv('washington_hikes.csv', encoding='utf-8')
            print ('Collected %d hike descriptions' % counter)
    data.to_csv('washington_hikes.csv', encoding='utf-8')


# if __name__ == '__main__':
#     urls = collect_hikeurls('http://www.wta.org/go-hiking/hikes')
#     data = pd.read_csv('washington_hikes.csv')
#     build_dataset(data, urls)
#     getHikeDescription(data)

Collected 30 websites


FileNotFoundError: File b'washington_hikes.csv' does not exist

In [7]:
urls = collect_hikeurls('http://www.wta.org/go-hiking/hikes')

Collected 30 websites


In [9]:
data = pd.read_csv('washington_hikes.csv')
build_dataset(data, urls)

EmptyDataError: No columns to parse from file

In [36]:
def collect_hikeurls(starturl, params=None):
    """Collecting all of the websites for all of the hikes"""
    hike_links = []
    path = starturl
    counter = 0
    while True:
        r = requests.get(path)
        soup = BeautifulSoup(r.text, 'lxml')
        for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
            hike_links.append(div['href'])
            counter += 1
        print ('Collected %d websites' % counter)
        link = soup.find('span', attrs={'class': 'next'})
        if link is None:
            break
        else:
            path = link.a['href']
    return hike_links 

In [10]:
url = 'http://www.wta.org/go-hiking/hikes'

In [46]:
def get_all_hike_urls(starturl, params=None):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
        hike_urls.append(div['href'])

In [45]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
            print(div['href'])
trail_count = soup.select_one('span.search-count').text
count = int("".join(filter(str.isdigit, trail_count)))
# .attrs.get('span')
# for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
#     print(div['href'])

https://www.wta.org/go-hiking/hikes/maple-hollow-park
https://www.wta.org/go-hiking/hikes/point-no-point
https://www.wta.org/go-hiking/hikes/mary-e-theler-wetlands-nature-preserve
https://www.wta.org/go-hiking/hikes/guillemot-cove
https://www.wta.org/go-hiking/hikes/kopachuck-state-park-trail
https://www.wta.org/go-hiking/hikes/otter-falls-taylor-river
https://www.wta.org/go-hiking/hikes/sedum-point
https://www.wta.org/go-hiking/hikes/greenwater-lakes-quinn-lost-lakes
https://www.wta.org/go-hiking/hikes/umtanum-ridge-crest
https://www.wta.org/go-hiking/hikes/narbeck-wetland-sanctuary
https://www.wta.org/go-hiking/hikes/skookum-flats
https://www.wta.org/go-hiking/hikes/baldy-mountain-1
https://www.wta.org/go-hiking/hikes/wenaha-river-trail-3106
https://www.wta.org/go-hiking/hikes/icicle-ridge
https://www.wta.org/go-hiking/hikes/black-hole-falls-hike
https://www.wta.org/go-hiking/hikes/grouse-mountain-n-fork-umatilla-wilderness
https://www.wta.org/go-hiking/hikes/bennington-lake
https://

In [35]:
count

3495

In [75]:
def iterate_all_reports(starturl):
    """Determines the number of hikes listed on the WTA trails page and get said number of report urls
    **Input parameters**
    ------------------------------------------------------------------------------
    starturl: string. Base URL for the request.
    **Output**
    ------------------------------------------------------------------------------
    None. Runs collect_hikeurls and saves all urls
    """
    hike_urls = []
    counter = 0
    r = requests.get(starturl).text
    soup = BeautifulSoup(r, 'lxml')
    trail_count = soup.select_one('span.search-count').text
    count = int("".join(filter(str.isdigit, trail_count)))
    numit = math.ceil(float(count)/30)
    for i in range(int(numit)):
        spot = f'b_start:int= {str(i*30)}'
        url = starturl+spot
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        print
        for div in soup.findAll('a', attrs={'class': 'listitem-title'}):
            hike_urls.append(div['href'])
            counter += 1
        print ('Collected %d websites' % counter)
    return hike_urls

In [73]:
urls = iterate_all_reports('https://www.wta.org/go-outside/hikes?')

b_start:int= 0
Collected 30 websites
b_start:int= 30
Collected 60 websites
b_start:int= 60
Collected 90 websites
b_start:int= 90
Collected 120 websites
b_start:int= 120
Collected 150 websites
b_start:int= 150
Collected 180 websites
b_start:int= 180
Collected 210 websites
b_start:int= 210
Collected 240 websites
b_start:int= 240
Collected 270 websites
b_start:int= 270
Collected 300 websites
b_start:int= 300
Collected 330 websites
b_start:int= 330
Collected 360 websites
b_start:int= 360
Collected 390 websites
b_start:int= 390
Collected 420 websites
b_start:int= 420
Collected 450 websites
b_start:int= 450
Collected 480 websites
b_start:int= 480
Collected 510 websites
b_start:int= 510
Collected 540 websites
b_start:int= 540
Collected 570 websites
b_start:int= 570
Collected 600 websites
b_start:int= 600
Collected 630 websites
b_start:int= 630
Collected 660 websites
b_start:int= 660
Collected 690 websites
b_start:int= 690
Collected 720 websites
b_start:int= 720
Collected 750 websites
b_start:

In [74]:
urls

['https://www.wta.org/go-hiking/hikes/maple-hollow-park',
 'https://www.wta.org/go-hiking/hikes/point-no-point',
 'https://www.wta.org/go-hiking/hikes/mary-e-theler-wetlands-nature-preserve',
 'https://www.wta.org/go-hiking/hikes/guillemot-cove',
 'https://www.wta.org/go-hiking/hikes/kopachuck-state-park-trail',
 'https://www.wta.org/go-hiking/hikes/otter-falls-taylor-river',
 'https://www.wta.org/go-hiking/hikes/sedum-point',
 'https://www.wta.org/go-hiking/hikes/greenwater-lakes-quinn-lost-lakes',
 'https://www.wta.org/go-hiking/hikes/umtanum-ridge-crest',
 'https://www.wta.org/go-hiking/hikes/narbeck-wetland-sanctuary',
 'https://www.wta.org/go-hiking/hikes/skookum-flats',
 'https://www.wta.org/go-hiking/hikes/baldy-mountain-1',
 'https://www.wta.org/go-hiking/hikes/wenaha-river-trail-3106',
 'https://www.wta.org/go-hiking/hikes/icicle-ridge',
 'https://www.wta.org/go-hiking/hikes/black-hole-falls-hike',
 'https://www.wta.org/go-hiking/hikes/grouse-mountain-n-fork-umatilla-wildernes